In [2]:
import json
import numpy as np
import pandas as pd 
import os

player_json_file_path = os.path.join('Resources', 'combined_player_tags.json') 

with open(player_json_file_path) as fp:
    player_data = json.load(fp)

In [158]:
type(player_data) 
# Player data is dictionary nested list

list

In [3]:
df = pd.json_normalize(player_data) 
df.head()

,player_tag,data.tag,data.name,data.expLevel,data.trophies,data.bestTrophies,data.wins,data.losses,data.battleCount,data.threeCrownWins,...,data.lastPathOfLegendSeasonResult.rank,data.bestPathOfLegendSeasonResult.leagueNumber,data.bestPathOfLegendSeasonResult.trophies,data.bestPathOfLegendSeasonResult.rank,data.currentFavouriteCard.maxEvolutionLevel,data.currentFavouriteCard.iconUrls.evolutionMedium,data.leagueStatistics.previousSeason.id,data.leagueStatistics.previousSeason.trophies,data.leagueStatistics.previousSeason.bestTrophies,data.leagueStatistics.bestSeason.rank
0,%23LV0LV,#LV0LV,FAJRI JR,5,676,857,73,97,173,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,%238R80YP,#8R80YP,<c7>omeganike</,37,5492,5671,3382,3359,7821,1193,...,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,%23R0VJQ,#R0VJQ,jonas123,10,1954,2235,405,437,886,263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,%23JRRULRR9C,#JRRULRR9C,samuel,4,377,423,25,30,56,9,...,NaN,NaN,NaN,NaN,1.0,https://api-assets.clashroyale.com/cardevoluti...,NaN,NaN,NaN,NaN
4,%23QRJVULQQ9,#QRJVULQQ9,ash,8,3775,3848,155,153,309,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
# Total number of members in this random sample 
len(player_data)

9390

In [5]:
# First get the player level data without the nested dictionaries
# Initialize key dataframes with columns
player_data[1]['data'].keys()
# The first dataframe is player level data
player_master_column = list(player_data[1]['data'].keys())[:18]
player_master_column.insert(0,'player_tag')    
player_master_column.append('totalExpPoints')
player_master_column.append('legacyTrophyRoadHighScore')
player_master_column.append('expPoints')

player_master = pd.DataFrame(columns=player_master_column)

for i in range(0, len(player_data)):
    # Add a new row
    row = [player_data[i]['player_tag']]
    for j in player_master_column[1:]:
        row.append(player_data[i]['data'][j])
    player_master = pd.concat([pd.DataFrame([row], columns=player_master.columns), player_master], ignore_index=True)

player_master.head() 

,player_tag,tag,name,expLevel,trophies,bestTrophies,wins,losses,battleCount,threeCrownWins,...,tournamentCardsWon,tournamentBattleCount,donations,donationsReceived,totalDonations,warDayWins,clanCardsCollected,totalExpPoints,legacyTrophyRoadHighScore,expPoints
0,%23L9QG080Q9,#L9QG080Q9,luca2724,27,5117,5285,416,272,694,181,...,0,21,0,0,431,0,0,26478,5285,3708
1,%23P8CL9JVUQ,#P8CL9JVUQ,mv,3,278,278,12,16,29,8,...,0,0,0,0,0,0,0,0,None,77
2,%23GYY8V,#GYY8V,Kingaj282,25,3800,4076,930,971,2173,521,...,0,8,0,0,2100,1,3465,17843,4076,2073
3,%2390CCQLL,#90CCQLL,Bodzio,9,2857,2982,444,348,1223,235,...,0,0,0,0,2478,0,0,14174,None,5404
4,%23U8QYJ92L9,#U8QYJ92L9,lin,2,150,150,5,0,5,5,...,0,0,0,0,0,0,0,58,None,38


In [8]:
# Drop duplicates
player_master = player_master.drop_duplicates()
player_master.to_csv( os.path.join('Resources', 'player_master.csv') )

In [9]:
# Function for dropping duplicates
def normalize_json_df(col):
    df_column = list(player_data[1]['data'][col].keys())
    df_column.insert(0,'player_tag')    
    # Initialize key dataframes with columns
    df = pd.DataFrame(columns=df_column)

    for i in range(0, len(player_data)):
        if  type(player_data[i]['data'][col]) is dict:
            # Add a new row
            row = [player_data[i]['player_tag']]
            for j in df_column[1:]:
                if player_data[i]['data'][col][j] is None:
                    row.append(np.nan)
                else:
                    row.append(player_data[i]['data'][col][j])
            df = pd.concat([pd.DataFrame([row], columns=df.columns), df], ignore_index=True)
    print(df.head()     )
    
    return(df.drop_duplicates())

In [10]:
arena_master = normalize_json_df('arena')

     player_tag        id      name
0  %23L9QG080Q9  54000012  Arena 15
1  %23P8CL9JVUQ  54000001   Arena 1
2      %23GYY8V  54000011  Arena 12
3    %2390CCQLL  54000010   Arena 9
4  %23U8QYJ92L9  54000001   Arena 1


In [11]:
bestPathOfLegendSeasonResult_master = normalize_json_df('bestPathOfLegendSeasonResult')

     player_tag leagueNumber trophies rank
0  %23L9QG080Q9            1        0  NaN
1     %23JRR90Y            4        0  NaN
2   %23YPLURQUJ            1        0  NaN
3    %23R9G8RLG            3        0  NaN
4     %23URJ8P2            4        0  NaN


In [12]:
currentPathOfLegendSeasonResult_master = normalize_json_df('currentPathOfLegendSeasonResult')

     player_tag leagueNumber trophies rank
0  %23L9QG080Q9            1        0  NaN
1     %23JRR90Y            1        0  NaN
2   %23YPLURQUJ            1        0  NaN
3    %23R9G8RLG            1        0  NaN
4     %23URJ8P2            1        0  NaN


In [13]:
lastPathOfLegendSeasonResult_master = normalize_json_df('lastPathOfLegendSeasonResult')

     player_tag leagueNumber trophies rank
0  %23L9QG080Q9            1        0  NaN
1     %23JRR90Y            1        0  NaN
2   %23YPLURQUJ            1        0  NaN
3    %23R9G8RLG            3        0  NaN
4     %23URJ8P2            1        0  NaN


In [17]:
def normalize_json_two_layers(col):
    master_column = list(player_data[1]['data'][col][0].keys())
    master_column.insert(0,'player_tag')    
    # Initialize key dataframes with columns
    df_master = pd.DataFrame(columns=master_column)

    for i in range(8, len(player_data)):
        if len(player_data[i]['data'][col]) > 0 :
            # print(i)
            for h in range(0, len(player_data[i]['data'][col])):
                # Add a new row
                # print(h)
                if  type(player_data[i]['data'][col][h]) is dict:      
                    row = [player_data[i]['player_tag']]
                    # print(row)
                    for j in master_column[1:]: 
                        # print(j)
                        if j in list(player_data[i]['data'][col][h].keys()):                                           
                            if player_data[i]['data'][col][h][j] is None:
                                row.append(np.nan)
                            else:
                                # print(player_data[i]['data']['badges'][h][j])
                                row.append(player_data[i]['data'][col][h][j])
                            # print(row)
                        else: 
                            row.append(np.nan)
                    df_master = pd.concat([pd.DataFrame([row], columns=df_master.columns), df_master], ignore_index=True)
    print(df_master.head() )
    return(df_master)

badges_master = normalize_json_two_layers('badges')
badges_master.to_csv(os.path.join('Resources', 'badges_master.csv'))

     player_tag                     name level maxLevel progress target  \
0  %23L9QG080Q9             MasteryTesla     1       10        1      2   
1  %23L9QG080Q9        MasteryRoyalGiant     1       10        1      2   
2  %23L9QG080Q9  MasterySkeletonWarriors     1       10        1      2   
3  %23L9QG080Q9       MasteryElixirGolem     1       10        1      2   
4  %23L9QG080Q9        MasteryEarthquake     1       10        1      2   

                                            iconUrls  
0  {'large': 'https://api-assets.clashroyale.com/...  
1  {'large': 'https://api-assets.clashroyale.com/...  
2  {'large': 'https://api-assets.clashroyale.com/...  
3  {'large': 'https://api-assets.clashroyale.com/...  
4  {'large': 'https://api-assets.clashroyale.com/...  


In [18]:
achievements_master = normalize_json_two_layers('achievements')
achievements_master.to_csv(os.path.join('Resources', 'achievements.csv'))

     player_tag                   name stars value target  \
0  %23L9QG080Q9      Friend in Need II     0   431   5000   
1  %23L9QG080Q9      Special Challenge     3    23      5   
2  %23L9QG080Q9  Practice with Friends     3    18     10   
3  %23L9QG080Q9       Challenge Streak     3    12     12   
4  %23L9QG080Q9      Tournament Player     3     2      1   

                                               info completionInfo  
0                                 Donate 5000 cards            NaN  
1  Participate in 5 unique Special Event Challenges            NaN  
2                           Win 10 Friendly Battles            NaN  
3                 Get 12 wins in a single Challenge            NaN  
4                                 Join a tournament            NaN  


In [ ]:
cards_master = normalize_json_two_layers('cards')
cards_master.to_csv(os.path.join('Resources', 'cards_master.csv'))

In [14]:
# Save files to csv
arena_master.to_csv(os.path.join('Resources', 'arena_master.csv') )   
bestPathOfLegendSeasonResult_master.to_csv(os.path.join('Resources', 'bestPathOfLegendSeasonResult_master.csv')  )   
currentPathOfLegendSeasonResult_master.to_csv(os.path.join('Resources', 'currentPathOfLegendSeasonResult_master.csv') )   
lastPathOfLegendSeasonResult_master.to_csv(os.path.join('Resources', 'lastPathOfLegendSeasonResult_master.csv') )   
badges_master.to_csv(os.path.join('Resources', 'badges_master.csv')   
achievements_master.to_csv(os.path.join('Resources', 'achievements_master.csv')   
cards_master.to_csv(os.path.join('Resources', 'cards_master.csv')   


In [ ]:
player_data[1]['data']['leagueStatistics']

{'currentSeason': {'trophies': 5492, 'bestTrophies': 5171},
 'bestSeason': {'id': '2021-09', 'trophies': 5618}}

In [230]:
# Initialize key dataframes with columns
player_data[1]['data']['starPoints']

46927